## Day 14: Rolling rocks!

**Part 1, calculate the load which is SUM[round rocks] { dist_from_south(rock_i) }**

```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```

becomes (+ distance numbers)

```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```

s.t. the total stress = 136

In [1]:
with open("./example.txt") as f:
    example_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

example_lines

['O....#....',
 'O.OO#....#',
 '.....##...',
 'OO.#O....O',
 '.O.....O#.',
 'O.#..O.#.#',
 '..O..#O..O',
 '.......O..',
 '#....###..',
 '#OO..#....']

In [2]:
from collections import deque

def shift_string(string: str, index: int, to_move_indexes: deque[int]):
    """
    To avoid sharing state, at first pass have to set index=0, to_move_indexes = deque([])
    """
    if index == len(string):
        if not isinstance(string, str):
            string = "".join(string)
            # cba to find out WHY it can even be a tuple here....   
        return string, index, to_move_indexes

    if string[index] == "#":
        to_move_indexes = deque([])
        return shift_string(string, index + 1, to_move_indexes)
    
    if string[index] == ".":
        to_move_indexes.append(index)
        return shift_string(string, index+1, to_move_indexes)
    
    if string[index] == "O":
        if len(to_move_indexes) > 0:
            # Moving O from index to move_to_index
            move_to_idx = to_move_indexes.popleft()
            string_as_list = [*string]

            string_as_list[index] = "."
            to_move_indexes.append(index)
            string_as_list[move_to_idx] = "O"

            string = "".join(string_as_list)
            return shift_string(string, index + 1, to_move_indexes)
        else:
            # leave it where it is then
            return shift_string(string, index + 1, to_move_indexes)

for string in [
    'OO.O.O..##',
    '...OO....O',
    '.O...#O..O',
    '.O.#......',
    '.#.O......',
    '#.#..O#.##',
    '..#...O.#.',
    '....O#.O#.',
    '....#.....',
    '.#.O.#O...',
    ]:
    tmp, _, _ = shift_string(string, 0, deque([]))
    print(string, tmp)

OO.O.O..## OOOO....##
...OO....O OOO.......
.O...#O..O O....#OO..
.O.#...... O..#......
.#.O...... .#O.......
#.#..O#.## #.#O..#.##
..#...O.#. ..#O....#.
....O#.O#. O....#O.#.
....#..... ....#.....
.#.O.#O... .#O..#O...


In [3]:
def part1(lines: list[str]):

    shifted_cols = [
        shift_string(col, 0, deque([]))[0] for col in zip(*lines)
    ]

    total = 0

    for col in shifted_cols:
        for i, ch in enumerate(col):
            if ch == "O":
                total += (len(col) - i)
    
    return total

assert part1(example_lines) == 136
%timeit part1(input_lines)

24.5 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Part 2: Oh god it's tilting NORTH, WEST, SOUTH, THEN EAST each cycle, and there are `1000000000` bloody cycle... What's the load on north after all that?**

In [4]:
start = [
        'O....#....',
        'O.OO#....#',
        '.....##...',
        'OO.#O....O',
        '.O.....O#.',
        'O.#..O.#.#',
        '..O..#O..O',
        '.......O..',
        '#....###..',
        '#OO..#....'
    ]
N = [
    "".join(row) for row in zip(*start)
]
W = ["".join(row) for row in zip(*N)]
S = ["".join(row[::-1]) for row in zip(*W)]
E = ["".join(row) for row in zip(*S)]
start = [row for row in E.__reversed__()]
start

['O....#....',
 'O.OO#....#',
 '.....##...',
 'OO.#O....O',
 '.O.....O#.',
 'O.#..O.#.#',
 '..O..#O..O',
 '.......O..',
 '#....###..',
 '#OO..#....']

In [80]:
from copy import deepcopy

def part2(lines: list[str]):
    def cycle(lines: list[str]):
        # TO NORTH
        shifted_cols = [
            shift_string(col, 0, deque([]))[0] for col in zip(*lines)
        ]
        # TO WEST
        shifted_cols = [
            shift_string(col, 0, deque([]))[0] for col in zip(*shifted_cols)
        ]
        
        # At this point the orientation is all good.
        # TO SOUTH
        shifted_cols = [
            shift_string(col[::-1], 0, deque([]))[0] for col in zip(*shifted_cols)
        ]

        # # TO EAST
        shifted_cols = [
            shift_string(col[::-1], 0, deque([]))[0][::-1] for col in zip(*shifted_cols)
        ]
        shifted_cols.reverse()
        # orientation is as at start

        return shifted_cols
    

    states = {}

    lines_after_one_cycle = cycle(lines)

    # HAVE VERIFIED CYCLE WORKS IN LINE WITH EXAMPLE !
    # lines_after_one_cycle = cycle(lines_after_one_cycle)
    # lines_after_one_cycle = cycle(lines_after_one_cycle)
    # return (["".join(x) for x in lines_after_one_cycle])


    string_to_compare = "".join(lines_after_one_cycle)

    states[string_to_compare] = (1, deepcopy(lines_after_one_cycle))

    cycles = 1
    input_to_alter = deepcopy(lines_after_one_cycle)
    while True:
        cycles += 1
        input_to_alter = cycle(input_to_alter)
        comparison_string = "".join(input_to_alter)
        if comparison_string in states:
            cycle_idx_1, cycle_val_1 = states[comparison_string]
            assert comparison_string == "".join(cycle_val_1)
            cycle_idx_2 = cycles
            # print(f"We found that equality with after {states[string_to_compare]} cycles, and after {cycles} cycles")
            break
        else:
            states[comparison_string] = (cycles, deepcopy(input_to_alter))

        if cycles > 1000000:
            # give it a mil I guess
            print("over a mil cycles, let's give up?")
            return 0

    left_to_do_at_end_of_known_cycle = 1e9 - cycle_idx_2

    left_to_do_at_end_of_final_possible_cycle = left_to_do_at_end_of_known_cycle % (cycle_idx_2 - cycle_idx_1)

    # we know that it repeates every cycle_idx_2 - cycle_idx_1
    # so skip them by doing modulo

    final_lines = cycle_val_1
    
    for _ in range(int(left_to_do_at_end_of_final_possible_cycle)):
        final_lines = cycle(final_lines)

    
    shifted_cols = ["".join(col) for col in zip(*final_lines)]
    total = 0
    for col in shifted_cols:
        for i, ch in enumerate(col):
            if ch == "O":
                total += (len(col) - i)
    
    return total
    

assert part2(example_lines) == 64
part2(input_lines)  # 118752 isn't right

118747

woo not too slow! Hardest bit was thinking about NESW tilting in a cycle lol, trying to keep orientation 'correct' at least by the end

In [25]:
['.....#....',
 '....#.O..#',
 'O..O.##...',
 'O.O#......',
 'O.O....O#.',
 'O.#..O.#.#',
 'O....#....',
 'OO....OO..',
 '#O...###..',
 '#O..O#....']  # south

['.....#....',
 '....#.O..#',
 'O..O.##...',
 'O.O#......',
 'O.O....O#.',
 'O.#..O.#.#',
 'O....#....',
 'OO....OO..',
 '#O...###..',
 '#O..O#....']

In [6]:
1000000000 % 4

0

In [7]:
for i in range(0):
    print(i)

In [73]:
90 % 7

6

In [71]:
10 + 7*12

94